In [ ]:
import pandas as pd
import numpy as np
from os import system

import tensorflow as tf
import keras
from scipy import ndimage
from keras.preprocessing import image
from keras.models import Sequential
from keras.applications.vgg19 import preprocess_input

from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import datetime

%load_ext tensorboard

In [ ]:
system("unzip test.zip")
system("unzip train.zip")

In [ ]:
train_path = "train"
test_path = "test"
classes = pd.read_csv("drive/My Drive/label.csv", header=None, squeeze=True).tolist()[1:]

In [ ]:
train_batches = ImageDataGenerator(horizontal_flip=True).flow_from_directory(train_path, target_size = (224,224), classes=classes, batch_size = 50)
test_batches = ImageDataGenerator(horizontal_flip=True).flow_from_directory(test_path, target_size = (224,224), classes=classes, batch_size = 50)

In [ ]:
vgg_19 = keras.applications.vgg19.VGG19()
mod = Sequential()
for layer in vgg_19.layers:
    mod.add(layer)
for layer in mod.layers:
    layer.trainable = False
mod.layers.pop()
mod.add(Dense(355, activation="softmax"))
mod.compile(Adam(lr=.00005), loss="categorical_crossentropy",metrics=[tf.keras.metrics.TopKCategoricalAccuracy(k=3), "accuracy"])

In [ ]:
!rm -rf ./logs/
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
mod.fit(train_batches, validation_data =  test_batches, epochs = 7, callbacks=[tensorboard_callback], verbose =1)

In [ ]:
%tensorboard --logdir logs/
mod.save("full1")